# Rice Type Prediction

I will write a model that can predict types of rice. I am going to use a CNN model. But first, I need to download the data and then I will prepare it to train the model. End of the project I will make a streamlit app with my model that can predict types of rice. You can find the data in Kaggle.

In [ ]:
#I am importing libraries
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
#I am unziping the dataset
from google.colab import files
import zipfile

uploaded = files.upload()

for zip_filename in uploaded.keys():
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('Rice_Image_Dataset')


Saving archive.zip to archive.zip


In [ ]:
#My dataset has 5 classes
labels=["Arborio", "Basmati", "Ipsala", "Jasmine", "Karacadag"]
img_path="/content/Rice_Image_Dataset/Rice_Image_Dataset/"

In [ ]:
#This is a for loop that creates a list of images and labels
img_list=[]
label_list=[]
for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

In [ ]:
#I am transforming the list to dataframe
df=pd.DataFrame({"image":img_list,"label":label_list})
df.head()

,image,label
0,/content/Rice_Image_Dataset/Rice_Image_Dataset...,Arborio
1,/content/Rice_Image_Dataset/Rice_Image_Dataset...,Arborio
2,/content/Rice_Image_Dataset/Rice_Image_Dataset...,Arborio
3,/content/Rice_Image_Dataset/Rice_Image_Dataset...,Arborio
4,/content/Rice_Image_Dataset/Rice_Image_Dataset...,Arborio


In [ ]:
#I am encoding the labels
d={"Arborio":0,"Basmati":1,"Ipsala":2,"Jasmine":3,"Karacadag":4}

In [ ]:
df["encoded_label"]=df["label"].map(d)

In [ ]:
##!!!This step is very important!!!##
x=[]
for img in df["image"]:
    img=cv2.imread(str(img)) #That read the image
    img=cv2.resize(img,(170,170)) #That resize the image to 170x170
    img=img/255.0 #That normalize the image-->Normalizing makes the rgb values between 0 and 1. That facilitates our computers to process
    x.append(img)

In [ ]:
#I have to convert the list to array because keras model takes array
x=np.array(x)

In [ ]:
#My target is encoded_label
y=df["encoded_label"]

In [ ]:
#I am splitting my dataset-->x_train,x_test,y_train,y_test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization

In [ ]:
#I am creating my model
#I am using sequential model
#In my model I am using 5 convolutional layers
#I am using 3 pooling layers
#I am using 2 dense layers
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(80,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
1875/1875 [==============================] - 350s 186ms/step - loss: 0.1268 - accuracy: 0.9563 - val_loss: 0.0663 - val_accuracy: 0.9775
Epoch 2/10
1875/1875 [==============================] - 330s 176ms/step - loss: 0.0415 - accuracy: 0.9865 - val_loss: 0.0312 - val_accuracy: 0.9905
Epoch 3/10
1875/1875 [==============================] - 335s 178ms/step - loss: 0.0264 - accuracy: 0.9916 - val_loss: 0.0426 - val_accuracy: 0.9869
Epoch 4/10
1875/1875 [==============================] - 337s 180ms/step - loss: 0.0223 - accuracy: 0.9932 - val_loss: 0.0284 - val_accuracy: 0.9917
Epoch 5/10
1875/1875 [==============================] - 340s 181ms/step - loss: 0.0194 - accuracy: 0.9938 - val_loss: 0.0158 - val_accuracy: 0.9953
Epoch 6/10
1875/1875 [==============================] - 336s 179ms/step - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.0235 - val_accuracy: 0.9931
Epoch 7/10
1875/1875 [==============================] - 343s 183ms/step - loss: 0.0164 - accuracy: 0.9949 - val_

In [ ]:
#I am saving my model for making my streamlit app
model.save("my_rice_prediction_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


To conclusion I can say that the model is very good. Its accuracy is 99.63%. In that project I used 15000 images per class 75000 images in total. My computer hasn't got enough memory to run that model. So I used Google Colab. But you don't have to use Colab. You can take enough samples that your computer memory can handle with that code: df.sample(how_many_samples). You can see my streamlit app on [Hugging Face](https://huggingface.co/spaces/emirkaanozdemr/predict_rice_types/tree/main).